# 🟩 나의 커스텀 데이터셋 만들기  

<br>

## 🟢 커스템 데이터셋 클래스 만들기  



In [ ]:
from torch.utils.data import Dataset  # 커스텀 데이터셋을 만들기 위함
import os
import cv2
import numpy as np

class CustomData(Dataset):
    def __init__(self, path, transform):
        self.video_path = path 
        self.transform = transform 

    def __len__(self):
        # if (이미지수 == 라벨수) return 이미지 수
        return len(self.video_path)
        # __len__은 DataLoader가 전체 데이터 개수를 알기 위해 호출합니다.

    def same_fps(self, path):  # 하나의 동영상 경로만 들어가서 처리합니다.
        video = cv2.VideoCapture(path)
        if not video.isOpened():
            return print("Could not Open :", path)

        target_fps = 30 

        original_fps = int(video.get(cv2.CAP_PROP_FPS))
        print(f"현재 초당 프레임 수 : {original_fps}")

        frame_length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f"현재 총 프레임 수 : {frame_length}")

        frames = []
        count = 0

        while True:
            ret, img = video.read()  # 한 프레임 읽기
            if not ret:  # 더 이상 프레임 없으면 종료
                break

            if original_fps > target_fps:
                step = round(original_fps / target_fps) 
                if count % step == 0:  
                    frames.append(img)
            elif original_fps < target_fps:
                frames.append(img) 
                if (count + 1) % original_fps == 0:
                    frames.append(img)  
            else:
                frames.append(img)

            count += 1  # 읽은 프레임 수 증가

        video.release()  

        return np.array(frames)

    # 텐서 형태로 변환된 이미지를 돌려줌
    def __getitem__(self, idx):
        human_accident_class_list = [
            "bump",
            "fall-down",
            "fall-off",
            "hit",
            "jam",
            "no-accident"
        ]   # class에 대한 list를 정의하였습니다.

        video = self.video_path[idx]  # 하나의 영상 경로를 얻음

        # dirname = 마지막 파일 이름을 제거하고 상위 폴더 경로만 남긴 뒤 / basename = 맨 마지막 부분(폴더명 또는 파일명) 만 뽑아냅니다.
        human_accident_class_name = os.path.basename(os.path.dirname(video))

        video = self.same_fps(video)  # 하나의 영상 fps를 30프레임에 맞춰서 넘파이 배열 반환

        if self.transform is not None:
            result = self.transform(video)

        # 현재 영상에 대한 class를 index(번호)로 가져가 label로 붙여줍니다.
        label = human_accident_class_list.index(human_accident_class_name)

        return result, label

<br>

## 🟢 class별 동영상 path 지정 (`동영상별 path` list가 만듬)  

In [19]:
import os  # 로컬 컴퓨터에서 파일 및 폴더 컨트롤 라이브러리

origin = "C:\\Users\\bb\\Desktop\\learn_deeplearning\\human-accident\\data\\safety-data\\human-accident\\"

human_accident_class = os.listdir(origin)
print(human_accident_class)

video_path = []

for accident_class in human_accident_class:
    for x in os.listdir(origin + accident_class):
        video_path.append(os.path.join(origin, accident_class, x))

print(video_path)

['bump', 'fall-down', 'fall-off', 'hit', 'jam', 'no-accident']
['C:\\Users\\bb\\Desktop\\learn_deeplearning\\human-accident\\data\\safety-data\\human-accident\\bump\\human-accident_bump_rgb_0001_cctv1.avi', 'C:\\Users\\bb\\Desktop\\learn_deeplearning\\human-accident\\data\\safety-data\\human-accident\\bump\\human-accident_bump_rgb_0005_cctv2.avi', 'C:\\Users\\bb\\Desktop\\learn_deeplearning\\human-accident\\data\\safety-data\\human-accident\\bump\\human-accident_bump_rgb_0006_cctv2.avi', 'C:\\Users\\bb\\Desktop\\learn_deeplearning\\human-accident\\data\\safety-data\\human-accident\\bump\\human-accident_bump_rgb_0021_cctv2.avi', 'C:\\Users\\bb\\Desktop\\learn_deeplearning\\human-accident\\data\\safety-data\\human-accident\\bump\\human-accident_bump_rgb_0047_cctv3.avi', 'C:\\Users\\bb\\Desktop\\learn_deeplearning\\human-accident\\data\\safety-data\\human-accident\\bump\\human-accident_bump_rgb_0049_cctv4.avi', 'C:\\Users\\bb\\Desktop\\learn_deeplearning\\human-accident\\data\\safety-data

<br>

## 🟢 transforms (전처리 방식)  

In [7]:
from torch.utils.data import DataLoader  # 데이터로더를 만들기 위함

# torchvision: PyTorch에서 이미지 처리/데이터셋 관련 기능을 제공하는 라이브러리
# 🔥transforms(전처리): 이미지의 크기를 조절하거나, 딥러닝 모델이 이해할 수 있는 텐서(Tensor) 형태로 변환하는 등 (자르기, 회전, 텐서 변환, 정규화 등)
#                       이미지 전처리(preprocessing) 기능을 담당합니다.
from torchvision import transforms

# 여러 가지 전처리 방법을 하나로 묶어주는 transforms.Compose를 사용합니다.
transform = transforms.Compose(
    [
        # 1. 이미지 크기 조절: 모든 이미지의 크기를 224x224 픽셀로 맞춥니다.
        transforms.Resize((224, 224)),
        # 2. 텐서(Tensor)로 변환: 이미지를 딥러닝 모델이 계산할 수 있는 숫자 행렬(텐서)로 바꿉니다.
        transforms.ToTensor(),
        # 3. 정규화(Normalize): 이미지의 픽셀 값 범위를 조정하여 모델이 더 빠르고 안정적으로 학습하도록 돕습니다.
        #    아래 mean과 std 값은 ImageNet 데이터셋에서 미리 계산된 값으로, 보통 그대로 많이 사용합니다.
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

<br><br>

---

## 🟢 사용  

In [ ]:
# 이미지에 대한 경로, 일괄적으로 적용할 전처리 이 2가지를 인수로 넣어줌.
dataset = CustomData(video_path, transform)



dataloader = DataLoader(
    dataset=dataset,
    batch_size=25,
    shuffle=True,
    drop_last=False,  # 마지막에 남는 데이터도 사용합니다. (False: 버리지 않음)

    # time
)